In [63]:
import pygame
import random
import numpy as np

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [64]:
pygame.init()

(6, 0)

In [65]:
pixell_array = 16777215

In [66]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
from torch.autograd import variable

In [67]:
input_size = 2
output_size = 8

In [68]:
class Net(nn.Module):
    def __init__(self,input_channel,output_channel):
        self.input_channel = input_channel
        self.output_channel = output_channel
        super(Net,self).__init__()
        
        self.conv1 = nn.Conv2d(input_channel,32,3,padding=0)
        self.batch_norm1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(32,32,3,padding=0)
        self.batch_norm2 = nn.BatchNorm2d(32)
        self.relu2 = nn.ReLU()
        
        self.fc1 = nn.Linear(1229312,output_channel)
        
    def forward(self,x):
        x = self.relu1(self.batch_norm1(self.conv1(x)))
        x = self.relu2(self.batch_norm2(self.conv2(x)))
        x = x.view(-1,1229312)
        x = self.fc1(x)
        return x

In [69]:
net1 = Net(input_size,output_size)
net2 = Net(input_size,output_size)
sm = nn.Softmax(dim=1)

In [70]:
def up(x,y):
    y-= 10
    return x,y
    
def down(x,y):
    y+= 10
    return x,y

def left(x,y):
    x -= 10
    return x,y

def right(x,y):
    x += 10
    return x,y

def upright(x,y):
    x += 10
    y -= 10
    return x,y

def upleft(x,y):
    x -= 10
    y -= 10
    return x,y

def downright(x,y):
    x += 10
    y += 10
    return x,y

def downleft(x,y):
    x -= 10
    y += 10
    return x,y

In [71]:
moves = (up,down,left,right,upright,upleft,downright,downleft)

In [72]:
def move_array(move):
    one = moves.index(move)
    array = [0]*8
    array[one] = 1
    return array

In [140]:
def move_indx(m):
    return moves.index(m)

In [142]:
def move_reverser(m):
    if m == up:
        return down
    if m == down:
        return up
    if m == left:
        return right
    if m == right:
        return left
    if m == upleft:
        return downright
    if m == downright:
        return upleft
    if m == upright:
        return downleft
    if m == downleft:
        return upright

In [ ]:
objective = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(params=net1.parameters(), lr=0.01)

In [166]:
Winners = []
episodes_batch = []
episodes_memory = [[],[]]
for z in range(100000):
    print("Episode No. ", z)
    red_moves = []
    black_moves = []
    states = []
    states_reversed = []
    area = []
    Won = ""
    fps = pygame.time.Clock()
    width = 200
    height = 200
    x = 180
    y = 25
    x_b = 25
    y_b = 180

    red = [255,0,0]
    white = [255,255,255]
    black = [0,0,0]
    win = pygame.display.set_mode((width,height))
    pygame.display.set_caption('First_world')
    win.fill(white)
    run = True
    while run:
        u = []
    #    x,y = random.choice(moves)(x,y)
    #    x_b,y_b = random.choice(moves)(x_b,y_b)
        if x > width-20:
            x=(width-20)
        if x < 20:
            x = (20)

        if y > height-20:
            y=(height-20)
        if y < 20:
            y = (20)

        if x_b > width-20:
            x_b=(width-20)
        if x_b < 20:
            x_b = (20)

        if y_b > height-20:
            y_b=(height-20)
        if y_b < 20:
            y_b = (20)



        pygame.draw.circle(win,red,(x,y),10)
        pygame.draw.circle(win,black,(x_b,y_b),10)
    #    pygame.draw.circle(win,red,(x+dx,y+dy),20)

        pygame.display.update()
    #    win.fill(white)
        fps.tick(60)

        for i in range(200):
            for j in range(200):
                u.append(win.get_at((i,j)))

        blacks = u.count((0, 0, 0, 255))
        reds = u.count((255, 0, 0, 255))

        area.append((blacks,reds))

        im_win = pygame.PixelArray(win)
        im_red = im_win.extract(pygame.Color(255, 0, 0, 255))
        im_red = np.array(im_red)/pixell_array
        im_black = im_win.extract(pygame.Color(0, 0, 0, 255))
        im_black = np.array(im_black)/pixell_array

        batch = torch.Tensor([im_red,im_black])
        batch= batch.view(-1,2,200,200)
        states.append(batch)
        move_probs1 = sm(net1(torch.FloatTensor(batch))).tolist()[0]

        probs1 = np.array(move_probs1)
        move1 = np.random.choice(moves,p=probs1/sum(probs1))
        red_moves.append(move1)
        x,y = move1(x,y)


        move_probs2 = sm(net2(torch.FloatTensor(batch))).tolist()[0]

        probs2 = np.array(move_probs2)
        move2 = np.random.choice(moves,p=probs2/sum(probs2))
        black_moves.append(move2)
        x_b,y_b = move2(x_b,y_b)


        if (area[-1][0]/sum(area[-1]) > 0.7) | (area[-1][1]/sum(area[-1]) > 0.7) :
            if (area[-1][0]/sum(area[-1]) > 0.7):
                print('Black ball won!!!')
                Won = 'Black'
                Winners.append(Won)
                run = False
            if (area[-1][1]/sum(area[-1]) > 0.7):
                print('Red ball won!!!')
                Won = 'Red'
                Winners.append(Won)
                run = False

    #    pygame.time.delay(20)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                
        if len(states) > 250:
            print("Tie")
            Won = "Tie"
            Winners.append(Won)
            run = False

    pygame.quit()
    if Won == 'Red':
        red_moves_array = list(map(move_indx,red_moves))
    #    black_moves_array = list(map(move_indx,black_moves))
        episodes_memory[0].extend([states])
        episodes_memory[1].extend([red_moves_array])
        if len(episodes_memory[0]) > 20000:
            del episodes_memory[0][:len(states)]
            del episodes_memory[1][:len(red_moves_array)]
    if len(episodes_memory[0]) > 5000:
        buff = random.sample(list(zip(episodes_memory[0],episodes_memory[1])),100)
        for i,j in buff:
            episodes_batch[0].append(i)
            episodes_batch[1].append(j)
        print("Training Red Ball!!!")
        train1 = []
        for i in episodes_batch[0]:
            move_probs = sm(net1(torch.FloatTensor(i))).tolist()[0]
            train1.append(move_probs)
        batch_size = len(episodes_batch[1])
        loss_v = objective(variable(torch.FloatTensor(train1), requires_grad=True).view(-1,8,batch_size),torch.LongTensor([episodes_batch[1]]))
        print("Loss is ",loss_v)
        optimizer1.zero_grad()
        loss_v.backward()
        optimizer1.step()
       

    if Won == 'Black':
        black_moves_reversed = list(map(move_reverser,black_moves))
        black_moves_array_reversed = list(map(move_indx,black_moves_reversed))
        for i in states:
            im_black_reversed = np.rot90(np.rot90(i.view(2,200,200)[0]))
            im_red_reversed = np.rot90(np.rot90(i.view(2,200,200)[1]))
            batch_reversed = torch.Tensor([im_red_reversed,im_black_reversed])
            batch_reversed.view(-1,2,200,200)
            states_reversed.append(batch_reversed)
            
        episodes_memory[0].extend([states_reversed])
        episodes_memory[1].extend([black_moves_array_reversed])
        if len(episodes_memory[0]) > 20000:
            del episodes_memory[0][:len(states_reversed)]
            del episodes_memory[1][:len(black_moves_array_reversed)]
            
        if len(episodes_memory[0]) > 5000:
            buff = random.sample(list(zip(episodes_memory[0],episodes_memory[1])),100)
            for i,j in buff:
                episodes_batch[0].append(i)
                episodes_batch[1].append(j)
            print("Training Red Ball!!!")
            train1 = []
            for i in episodes_batch[0]:
                move_probs = sm(net1(torch.FloatTensor(i))).tolist()[0]
                train1.append(move_probs)
            batch_size = len(episodes_batch[1])
            loss_v = objective(variable(torch.FloatTensor(train1), requires_grad=True).view(-1,8,batch_size),torch.LongTensor([episodes_batch[1]]))
            print("Loss is ",loss_v)
            optimizer1.zero_grad()
            loss_v.backward()
            optimizer1.step()

Episode No.  0
Red ball won!!!
Episode No.  1
Tie
Episode No.  2
Red ball won!!!
Episode No.  3
Red ball won!!!
Episode No.  4
Black ball won!!!
Episode No.  5
Red ball won!!!
Episode No.  6
Tie
Episode No.  7
Tie
Episode No.  8


KeyboardInterrupt: 

In [165]:
pygame.quit()

In [14]:
def win_stats():
    length = len(Winners)
    reds = Winners.count('Red')
    blacks = Winners.count('Black')
    ties = Winners.count('Tie')
    print("Total Episodes palyed: {0}, Reds Won: {1}, Blacks Won: {2}, Ties: {3}".format(length,reds,blacks,ties))

In [15]:
win_stats()

Total Episodes palyed: 23, Reds Won: 2, Blacks Won: 11, Ties: 10


In [ ]:
#loss = (torch.sum(torch.mul(torch.log_softmax(net1(batch),dim = 1).view(-1,8,11), variable(discounted_rewards)).mul(-1), -1))

In [2]:
import numpy as np

In [54]:
r = np.random.random_integers(93,size=(5,5))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: This function is deprecated. Please call randint(1, 93 + 1) instead
  """Entry point for launching an IPython kernel.


In [137]:
im_black_sw = np.rot90(np.rot90(im_black))

In [158]:
states[0].view(2,200,200)

tensor(1.)

In [162]:
for i in states:
    im_black_reversed = np.rot90(np.rot90(i.view(2,200,200)[0]))
    im_red_reversed = np.rot90(np.rot90(i.view(2,200,200)[1]))
    batch_reversed = torch.Tensor([im_red_reversed,im_black_reversed])
    batch_reversed.view(-1,2,200,200)
    states_reversed.append(batch_reversed)

In [163]:
im_red_reversed[0][175,25]

1.0

In [ ]:
batch = torch.Tensor([im_red,im_black])
batch= batch.view(-1,2,200,200)
states.append(batch)